In [1]:
import tensorflow as tf
import numpy as np

In [2]:
#Variables and Placeholders
a=tf.Variable([1.1,2,3,4,5])
b=tf.get_variable("b",shape=[3,5],initializer=tf.contrib.layers.xavier_initializer())
c=tf.Variable(tf.truncated_normal([10,13], stddev=0.1), name='c')

Ax=tf.placeholder(tf.float32, [None, 5], name='Ax')

In [3]:
#reuse
#get_variable可以创建variable和复用，是一个比较推荐的api
b=tf.get_variable("b",shape=[3,5],initializer=tf.contrib.layers.xavier_initializer())

ValueError: Variable b already exists, disallowed. Did you mean to set reuse=True in VarScope? Originally defined at:

  File "<ipython-input-2-7f06757f68f9>", line 3, in <module>
    b=tf.get_variable("b",shape=[3,5],initializer=tf.contrib.layers.xavier_initializer())
  File "/home/wk15/local/lib/python2.7/site-packages/IPython/core/interactiveshell.py", line 2885, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/home/wk15/local/lib/python2.7/site-packages/IPython/core/interactiveshell.py", line 2825, in run_ast_nodes
    if self.run_code(code, result):


In [5]:
#create interactive session
sess=tf.InteractiveSession()
tf.initialize_all_variables().run()

Instructions for updating:
Use `tf.global_variables_initializer` instead.


In [27]:
#run
#run整张图有两种，一种是eval,一种是sess.run 后者可以执行多个op： sess.run([a_op,b_op,c_op],feed_dict=dict_)
b.eval()

array([[-0.73853707, -0.16607672,  0.11512226, -0.45677897, -0.75502878],
       [ 0.2105369 ,  0.33206671, -0.6567589 ,  0.2938717 ,  0.27826434],
       [-0.55829459, -0.05030221, -0.43573695,  0.43425113, -0.27743405]], dtype=float32)

In [28]:
#feed_dict的值可以是list or np.array
#feed_dict里只要提供计算这个op需要代入placeholder的数据，tensorflow会自动在图中找到对应的路径进行计算，因此如果执行的是optimizer.minimize之类的op,更新梯度的时候也只更新这一部分
sess.run(Ax-a,feed_dict={Ax:[[20.1,31,55,61,99]]})
#返回值是np.array

array([[ 19.,  29.,  52.,  57.,  94.]], dtype=float32)

In [23]:
#shape
#静态得到shape的方法
(Ax-a).get_shape()

TensorShape([Dimension(None), Dimension(5)])

In [24]:
(Ax-a).get_shape()[1]

Dimension(5)

In [25]:
#强行转换成int
int((Ax-a).get_shape()[1])

5

In [26]:
#动态的得到，本身也是一个tensor，只有run时才能得到结果，但因为代入了placeholder，能得到placeholder中None的数值
tf.shape((Ax-a))

<tf.Tensor 'Shape_4:0' shape=(2,) dtype=int32>

In [20]:
sess.run(tf.shape((Ax-a)),feed_dict={Ax:[[19.3,19.2,21.5,23.8,5]]})

array([1, 5], dtype=int32)

In [34]:
#因为是tensor，本身也可以参与tensor的计算
#这里也用到了slides之类的奇技淫巧
sess.run(tf.gather(b,[tf.shape(Ax-a)[0],tf.shape(Ax-a)[1]-3]),feed_dict={Ax:[[19.3,19.2,21.5,23.8,5]]})
#gather和embedding_lookup的不同之处在于，后者的通常任务是在很多的向量中抽取一小部分，因此只更新这一部分梯度，优化效率

array([[ 0.2105369 ,  0.33206671, -0.6567589 ,  0.2938717 ,  0.27826434],
       [-0.55829459, -0.05030221, -0.43573695,  0.43425113, -0.27743405]], dtype=float32)

In [35]:
sess.run(b[1])

array([ 0.2105369 ,  0.33206671, -0.6567589 ,  0.2938717 ,  0.27826434], dtype=float32)

In [11]:
#broadcast
#规则参见numpy官网
#1.0版本的api向numpy靠拢，所以tensorflow用好以后会感觉numpy里面很多操作差不多
c=a-b
c.get_shape()

TensorShape([Dimension(3), Dimension(5)])

In [12]:
c.eval()

array([[ 1.8385371 ,  2.16607666,  2.88487768,  4.456779  ,  5.75502872],
       [ 0.88946313,  1.66793323,  3.65675879,  3.70612836,  4.72173548],
       [ 1.65829468,  2.05030227,  3.43573689,  3.56574893,  5.27743387]], dtype=float32)